<h1 style="text-align:center;">Churn Prediction</h1>

<h3> Before we begin, let me intoduce you to business problem.</h3>
A bank manager was disappointed as time to time many people have shut down their credit card services from that bank. and the bank manager wants to find out who is gonna leave the bank in future so he can give better offers to those customer to stick with them. so the bank manager collected the data for credit card services and wants to find answers.



<h2>So, How we gonna help the bank manager?</h2>

we will solve our problem in following steps(or how a data science project look like)

* collecting data
* data wrangling
* a deep dive analysis on data
* feature engineering part
* Data preprocessing
* building models
* hyperparameter tuning
* analyis on model


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing some useful libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.combine import SMOTETomek
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
#reading data from kaggle input
df=pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')
print('Toatal No oF rows',df.shape[0])
print('Toatal No oF columns',df.shape[1])
df.info()

As already mentioned in the problem statement that the last two columns are not useful at all. so we gonna drop it before proceeding to next step

In [ ]:
df.drop([df.columns[21],df.columns[22]],axis=1,inplace=True)
df.head(2)

Allright so we have deleted the last two columns and now we are done with data collecting process. now we will gain insight about data.

<h1 style="text-align:center;">Deep dive into data</h1>

In [ ]:
# lets describe the data.... the script written below will give info about many thing..lets check
df.describe()

**Note:** the above code only works on numerical data
<br>
As of now you can see that it gives the idea about mean ,max,count of the object.
before moving further lets check about the missing values in data.

In [ ]:
df.isnull().sum()

In [ ]:
sns.countplot(df['Attrition_Flag'])

IT is highly imbalanced dataset...as our target column is having 85-15 ration which needs a fine technuique to train our model.

AS we can see there are no missing values in data.. which is good for us.


<h1 style="text-align:center;">Exploratort data analysis</h1>

here we are splitting categorical data and numerical data..so that it will be easy for us to do anlysis.

In [ ]:
df_obj=df.select_dtypes(include=[object])
df_numr=df.select_dtypes(exclude=[object])

In [ ]:
df_obj

1. lets do UNIVARIATE analysis on categorical variable

In [ ]:
for i in df_obj.columns:
    print(df_obj[i].value_counts())
    




In [ ]:
# lets analyse it with the help of graph
fig, axes = plt.subplots(3, 2, figsize=(18, 10))

fig.suptitle('countplot')

sns.countplot(ax=axes[0, 0],x=df_obj['Gender'],hue=df_obj['Attrition_Flag'])
sns.countplot(ax=axes[0, 1], x=df_obj['Marital_Status'],hue=df_obj['Attrition_Flag'])
sns.countplot(ax=axes[1, 0], x=df_obj['Education_Level'],hue=df_obj['Attrition_Flag'])
sns.countplot(ax=axes[1, 1], x=df_obj['Income_Category'],hue=df_obj['Attrition_Flag'])
sns.countplot(ax=axes[2, 0], x=df_obj['Card_Category'],hue=df_obj['Attrition_Flag'])
sns.countplot(ax=axes[2, 1], x=df_obj['Attrition_Flag'],hue=df_obj['Attrition_Flag'])




from the above graph we can conclude the following points:
* females own more credit card than male
* Married people takes the more credit card
* mostly people are graduate 
* most people have salary less than 40k 
* almost 90% people have Blue Card

And last point is what we have to predict who will gonna leave services in Future.

In [ ]:

# dropping CLIENTNUM which is unique id so its not useful at all
df_numr.drop('CLIENTNUM',axis=1,inplace=True)
# now we gonna analyse numerical feature
df_numr.head()

In [ ]:
# now we will chevck if features are correlated
df_numr.corr()

# pretty difficult to understand..not a problem we create visualization for it
fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(18,10)

sns.heatmap(df_numr.corr(), annot = True, vmin=-1, vmax=1, center= 0, cmap= 'coolwarm', linewidths=3, linecolor='black',ax=ax)

# WHY correlation matrix
as we can see from above graph that many feature are highly correletaed to each other which means that increasing a also increase b... they have almost several feature. so it is important for us to drop highly correlated feature
which we gonna perform in feature selection process.

In [ ]:
#lets analyse some important numerical feature
fig, axes = plt.subplots(2, 2, figsize=(18, 10))

fig.suptitle('countplot')

sns.boxplot(ax=axes[0][0],y=df_numr['Customer_Age'])
sns.boxplot(ax=axes[0][1],y=df_numr['Dependent_count'])
sns.boxplot(ax=axes[1][0],y=df_numr['Credit_Limit'])
sns.boxplot(ax=axes[1][1],y=df_numr['Avg_Utilization_Ratio'])

**AS we can see from above that the data contains outliers in some feature... lets see hoe to solve this problem**

### So we are done with DAta Analysis phase.now we are ready to move further.

<h1 style="text-align:center;">Feature Engineering And Data Preprocessing</h1>

In [ ]:
# as we already know that some of the features in our dataset are highly correleated...so it is better practice
# for us to drop those feature
correlated_features = set()
correlation_matrix = df_numr.corr()
# we gonna drop feature which have pearson coffecirnts greater than 0.6
for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.6:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
            
df_numr.drop(labels=correlated_features, axis=1, inplace=True)

In [ ]:
# one hot encoding of object category data
obj_data=pd.get_dummies(df_obj.drop(columns=['Attrition_Flag']),drop_first=True)
obj_data

Allright pretty good we have reduced our feature.... to avoid the complex model...still it contains lots of feature.

In [ ]:
# joining our numr dataframe and object dataframe
main_df=pd.concat([obj_data,df_numr],axis=1)
main_df.head()

In [ ]:
# splitting dependent and independent feature 
Y=pd.get_dummies(df_obj['Attrition_Flag'],drop_first=True)
X=main_df

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
# before moving any further we have to deal with imbalanced dataset
# there are two tecnique to do handle imbalanced dataset..oversampling and undersampling
# if the data is too large we can perform down sampling... butr as our data is not so big we perform oversampling
smk=SMOTETomek(random_state=42)
xres,yres=smk.fit_sample(X,Y)

In [ ]:
print(xres.shape,yres.shape)

Allright, as you can see now that your dataset is balanced..we are ready to move further. **HAPPY!**

In [ ]:
# dividing dataset into training and testing set 
X_train, X_test, y_train, y_test = train_test_split(xres, yres, test_size=0.25, random_state=42)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

<h1 style="text-align:center;">MODEL BUILDING</h1>

model building is the easiest process in data science...there are lots of classification algorithm to go with...there is no certainity that this mdel or that model will perform model..
we have to try different model and see which perfortms better.
wer will go with RANDOM FOREST CLASSIFIER

In [ ]:
#we have trained our model with some random parameter and lets see the accuracy
classifier=RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=50)
classifier.fit(X_train,y_train)

In [ ]:
# calculating accuracy score and confusion matrix
prediction=classifier.predict(X_test)
print(accuracy_score(y_test,prediction))
print(confusion_matrix(y_test,prediction))

**as you can see we got a pretty decent score of 96% accuracy**
Lets tune hyperparameter to check if we increase our accuracy

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#param_grid = { 
#  'n_estimators': [10,50,200, 500],
#    'max_features': ['auto', 'sqrt', 'log2'],
#    'max_depth' : [4,5,6,7,8],
#   'criterion' :['gini', 'entropy']

#}
#rfc=RandomForestClassifier(random_state=42)
#CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
#CV_rfc.fit(X_train, y_train)

In [ ]:
#CV_rfc.best_params_

In [ ]:
# these parameters are obtained by hyperparameter tuning
classifierfinal=RandomForestClassifier(n_estimators=500,criterion='gini',random_state=50,max_depth=8,max_features='auto')
classifierfinal.fit(X_train,y_train)

In [ ]:

prediction1=classifierfinal.predict(X_test)
print(accuracy_score(y_test,prediction1))
print(confusion_matrix(y_test,prediction1))

wait a min**after tuning the model even we get less accuracy...sometimes it happens because initially our model have been overfitted the data...now this model is good to used **
<h3> also it depands on the domain expertise what kind of metrics they require</h3>

<h1> so finally we are done with how a datascience project look like in an industry.. if You find this notebook useful..please upvote it and leave a comment.</h1>